# AI-Powered Phishing Email Detection System

## Introduction
Phishing attacks are among the most prevalent forms of cyber threats, often relying on deceptive emails to trick recipients into revealing sensitive information or clicking malicious links. Traditional rule-based systems for phishing detection struggle to adapt to the rapidly evolving language and structure of phishing emails. As a result, artificial intelligence (AI) methods—particularly machine learning—have become essential tools for building more flexible and accurate detection systems.

In this project, we develop an AI-powered phishing email detection system that classifies emails as phishing or legitimate using natural language features and metadata. Our focus is on building a lightweight, interpretable prototype using the XGBoost classifier, a gradient boosting algorithm known for its performance and efficiency.

We use a publicly available dataset from Kaggle that includes both phishing and legitimate emails with labeled examples. You can access the dataset here:
[🔗 Phishing Email Dataset on Kaggle](https://www.kaggle.com/datasets/naserabdullahalam/phishing-email-dataset)

The project involves the following core components:
*   Data cleaning and feature extraction from email content.
*   Training and evaluation of an XGBoost classification model.
*   Applying explainability techniques (e.g., SHAP) to interpret model predictions.
*   Testing the model on real-world-like examples and documenting its strengths and limitations.

The goal is to create a simple, explainable, and effective prototype that could form the basis of a real-world email threat detection tool.

## Step 1: Environment set up

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import xgboost as xgb

## Step 2: Dataset Exploration

In [1]:
# Where 'text_combined' contains the text data and 'target' is 1 for phishing, 0 for legitimate

df = pd.read_csv('../dataset/phishing_email.csv')

# Check the dataset
print(df.head())
print(f"Dataset shape: {df.shape}")
print(f"Class distribution:\n{df['target'].value_counts()}")

NameError: name 'pd' is not defined

## Step 3: Split the data into training and testing sets

In [ ]:
X = df['text_combined']
y = df['target']

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print(f"Training set size: {X_train.shape[0]}")
print(f"Testing set size: {X_test.shape[0]}")